In [1]:
import numpy as np
import xgboost as xgb

In [2]:
from scipy.sparse import load_npz

In [3]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report

In [4]:
svd = TruncatedSVD(150)

In [5]:
xtrain = svd.fit_transform(load_npz('data/xtrain.npz'))
xtest = svd.transform(load_npz('data/xtest.npz'))

In [6]:
labels = np.load('data/label.npz')

In [ ]:
ytrain = labels['train'].reshape(-1, 1)
ytest = labels['test'].reshape(-1, 1)

In [ ]:
dtrain = xgb.DMatrix(xtrain, label=ytrain)
dtest = xgb.DMatrix(xtest, label=ytest)

In [ ]:
param = {'silent':1, 'objective':'binary:hinge', 'eval_metric':'auc'}
num_round = 100

In [8]:
bst = xgb.train(param, dtrain, num_round)
pred = bst.predict(dtest)
print(classification_report(ytest, pred))

             precision    recall  f1-score   support

          0       0.65      0.42      0.51      4707
          1       0.91      0.96      0.94     28288

avg / total       0.87      0.89      0.88     32995

